In [4]:
import datetime
import sqlite3
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib  # For saving the trained model

class RainwaterHarvestingSystem:
    def __init__(self, tank_capacity, catchment_area):
        """
        Initialize the rainwater harvesting system with ML integration.
        :param tank_capacity: Storage capacity of the tank (liters)
        :param catchment_area: Catchment area for water collection (sq. meters)
        """
        self.tank_capacity = tank_capacity
        self.catchment_area = catchment_area
        self.current_water_level = 0
        self.model = None  # Placeholder for the trained model

        # Database setup
        self.conn = sqlite3.connect('rainwater_harvesting.db')
        self.create_database()

    def create_database(self):
        """Create database tables for tracking rainfall and water usage"""
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS rainfall_data (
                date TEXT PRIMARY KEY,
                rainfall_mm REAL,
                collected_water REAL
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS water_usage (
                date TEXT,
                usage_liters REAL,
                purpose TEXT
            )
        ''')
        self.conn.commit()

    def calculate_rainfall_collection(self, rainfall_mm):
        """Calculate collected water based on rainfall and catchment area"""
        runoff_coefficient = 0.8
        collected_water = self.catchment_area * (rainfall_mm / 1000) * runoff_coefficient
        return collected_water

    def add_rainfall_data(self, date, rainfall_mm):
        """Record rainfall and update water level"""
        collected_water = self.calculate_rainfall_collection(rainfall_mm)
        self.current_water_level = min(self.current_water_level + collected_water, self.tank_capacity)

        cursor = self.conn.cursor()
        cursor.execute('''
            INSERT OR REPLACE INTO rainfall_data
            (date, rainfall_mm, collected_water)
            VALUES (?, ?, ?)
        ''', (date, rainfall_mm, collected_water))
        self.conn.commit()

    def use_water(self, date, usage_liters, purpose):
        """Record water usage"""
        if usage_liters <= self.current_water_level:
            self.current_water_level -= usage_liters

            cursor = self.conn.cursor()
            cursor.execute('''
                INSERT INTO water_usage
                (date, usage_liters, purpose)
                VALUES (?, ?, ?)
            ''', (date, usage_liters, purpose))
            self.conn.commit()
            return True
        return False

    def get_system_status(self):
        """Get current tank status"""
        return {
            'tank_capacity': self.tank_capacity,
            'current_water_level': self.current_water_level,
            'water_percentage': (self.current_water_level / self.tank_capacity) * 100
        }

    def train_rainfall_model(self, dataset_path):
        """Train an ML model for rainfall prediction using historical data"""
        df = pd.read_csv(dataset_path)

        # Convert 'date' to datetime and extract features
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day

        # Define features (X) and target (y)
        X = df[['year', 'month', 'day']]
        y = df['rainfall_mm']

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Save model
        joblib.dump(model, 'rainfall_model.pkl')
        self.model = model
        print("Rainfall prediction model trained and saved.")

    def predict_rainfall(self, year, month, day):
        """Predict rainfall for a given date using the trained model"""
        if self.model is None:
            self.model = joblib.load('rainfall_model.pkl')  # Load model if not already loaded

        X_new = np.array([[year, month, day]])
        predicted_rainfall = self.model.predict(X_new)[0]
        return predicted_rainfall

    def generate_monthly_report(self, year, month):
        """Generate a monthly report of rainfall and water usage"""
        cursor = self.conn.cursor()

        cursor.execute('''
            SELECT SUM(rainfall_mm), SUM(collected_water)
            FROM rainfall_data
            WHERE date LIKE ?
        ''', (f'{year}-{month:02d}%',))
        rainfall_stats = cursor.fetchone()

        cursor.execute('''
            SELECT SUM(usage_liters), purpose
            FROM water_usage
            WHERE date LIKE ?
            GROUP BY purpose
        ''', (f'{year}-{month:02d}%',))
        usage_stats = cursor.fetchall()

        return {
            'total_rainfall_mm': rainfall_stats[0] or 0,
            'total_collected_water': rainfall_stats[1] or 0,
            'water_usage_by_purpose': dict(usage_stats)
        }


# Example usage
def main():
    # Create a system with 5000L tank and 100 sq meters catchment
    rh_system = RainwaterHarvestingSystem(5000, 100)

    # Train the model (Provide dataset path)
    dataset_path = "rainfall_dataset.csv"
    rh_system.train_rainfall_model(dataset_path)

    # Predict rainfall
    predicted_rainfall = rh_system.predict_rainfall(2024, 4, 1)
    print(f"Predicted Rainfall for 2024-04-01: {predicted_rainfall:.2f} mm")

    # Simulate real data
    rh_system.add_rainfall_data('2024-03-15', 25.5)
    rh_system.use_water('2024-03-15', 500, 'Garden Irrigation')

    # Get system status
    status = rh_system.get_system_status()
    print(f"Current Water Level: {status['current_water_level']} L")
    print(f"Tank Percentage Full: {status['water_percentage']:.2f}%")

    # Generate report
    report = rh_system.generate_monthly_report(2024, 3)
    print("Monthly Report:", report)


if __name__ == "__main__":
    main()


Rainfall prediction model trained and saved.
Predicted Rainfall for 2024-04-01: 8.30 mm
Current Water Level: 2.04 L
Tank Percentage Full: 0.04%
Monthly Report: {'total_rainfall_mm': 25.5, 'total_collected_water': 2.04, 'water_usage_by_purpose': {}}


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [3]:
import pandas as pd

# Generate sample rainfall data
data = {
    "date": pd.date_range(start="2023-01-01", periods=365, freq="D").strftime('%Y-%m-%d'),
    "rainfall_mm": [round(abs(20 * ((i % 10) / 10))) for i in range(365)]  # Simulated rainfall pattern
}

# Create DataFrame
df = pd.DataFrame(data)

# Save as CSV
df.to_csv("rainfall_dataset.csv", index=False)

print("Dataset created: rainfall_dataset.csv")


Dataset created: rainfall_dataset.csv
